In [ ]:
import os


In [ ]:
import json
from langchain.retrievers import TFIDFRetriever
from langchain.docstore.document import Document

In [ ]:
def load_data(json_path):
    with open(json_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


In [ ]:
def prepare_documents(data):
    documents = []
    for case in data:
        # Combine all parts of clinical_case_summary into a single string
        clinical_summary = ''
        summary_dict = case.get('clinical_case_summary', {})
        for key, value in summary_dict.items():
            clinical_summary += f"{key}: {value}\n"
        
        doc = Document(
            page_content=clinical_summary,
            metadata=case  # Store the entire case as metadata
        )
        documents.append(doc)
    return documents


In [ ]:
def initialize_retriever(documents):
    retriever = TFIDFRetriever.from_documents(documents)
    return retriever


In [ ]:
def retrieve_similar_cases(retriever, query, k=3):
    results = retriever.get_relevant_documents(query)[:k]
    print(results[0])
    similar_cases = [doc.metadata for doc in results]
    return similar_cases


In [ ]:
json_file_path = 'neurology department.json'

data = load_data(json_file_path)

documents = prepare_documents(data)

In [ ]:
json_file_path = 'neurology department.json'

data = load_data(json_file_path)

documents = prepare_documents(data)

retriever = initialize_retriever(documents)

new_summary = """
Patient Basic Information: Elderly male, 70 years old.
Chief Complaint: Tremors in hands and slowed movements for 2 years, worsening over the last month.
Medical History: Diagnosed with hypertension 5 years ago, controlled with medication. No history of diabetes or stroke.
Physical Examination: Resting tremor observed in both hands, rigidity in limb muscles, bradykinesia noted during movement tests.
Imaging Examination: MRI shows mild atrophy in substantia nigra region.
Laboratory Examination: Normal blood work, slightly elevated cholesterol levels.
"""

new_summary="""
Patient Basic Information:Elderly female, 80 years old.,
Chief Complaint:Memory decline for 1 year, worsening for more than 10 days.,
Medical History:The patient's memory started to decline without any obvious cause a year ago, manifested as an inability to recall recent events. These symptoms have been progressing and worsening. More than 10 days ago, family members noticed a significant worsening of symptoms, accompanied by a decline in executive function. The patient has a history of hypertension, coronary heart disease, post-coronary stent implantation, and cerebral infarction.,
Physical Examination:Heart, lungs, and abdomen are normal. Neurological System: The patient is conscious, but reaction is slow, understanding and orientation are reduced, memory and calculation ability are decreased, speech is fluent, and she cooperates with the examination. Bilateral nasolabial folds are symmetrical, tongue protrudes in the middle, muscle tone and tendon reflexes of the four limbs are symmetrical and normal, muscle strength is grade V, sensation and coordinated movements are normal, bilateral Babinski sign is negative. Neck is soft, Kernig's sign is negative. NIHSS score: 0 points. Wartenberg's drinking water test: Grade 1. mRS score: 1 point. ADL score: 100 points.,
Imaging Examination:MRI scan: 1. Multiple infarctions in the bilateral semioval center, periventricular, basal ganglia, brainstem, and right cerebellum, and degeneration of brain white matter; 2. Abnormal signal in the left frontal plate, considering epidermoid cyst; 3. Bilateral hippocampal volume reduction; 4. Cerebral arteriosclerosis; 5. Multiple stenosis in the bilateral middle cerebral artery M1 segment, right middle cerebral artery M2 and M3 segments, and bilateral posterior cerebral artery P2 segments. Please combine with clinical history and related examinations.\n(2),
Laboratory Examination:Thyroid function test: Normal.\nCerebrospinal fluid test: 1. Color: Colorless; 2. Transparency: Slightly turbid; 3. Coagulability: Non-coagulating; 4. Pandy's test: Positive (+); 5. Cerebrospinal fluid protein (dry method) (csfpro) 737mg\/L \u2191; 6. Chloride (dry method) (CL) 118.0mmol\/L \u2193; 7. Cerebrospinal fluid immunoglobulin G (CSFIgG) 133.90mg\/L \u2191.\n(3)
"""

similar_cases = retrieve_similar_cases(retriever, new_summary, k=3)

# for idx, case in enumerate(similar_cases, 1):
#     print(f"\n--- Similar Case {idx} ---")
#     print(json.dumps(case, indent=4, ensure_ascii=False))

In [ ]:
def clean_case_data(cases):
    cleaned_cases = []
    for case in cases:
        cleaned_case = {
            "principal_diagnosis": case.get("principal_diagnosis", ""),
            "preliminary_diagnosis": case.get("preliminary_diagnosis", []),
            "diagnostic_basis": case.get("diagnostic_basis", []),
            "differential_diagnosis": case.get("differential_diagnosis", []),
            "clinical_case_summary": case.get("clinical_case_summary", {})
        }
        cleaned_cases.append(cleaned_case)
    return cleaned_cases

cleaned_similar_cases = clean_case_data(similar_cases)

for case in cleaned_similar_cases:
    print(json.dumps(case, indent=2))

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from typing import List
from langchain_openai import ChatOpenAI

In [ ]:
class DiagnosisReason(BaseModel):
    category: str = Field(description="The category of the examination, e.g., medical-history, Physical-Examination, Laboratory-Examination, Image-Examination")
    reasons: List[str] = Field(description="A list of precise reasons for the diagnosis based on the examination")

class FinalDiagnosis(BaseModel):
    final_diagnosis: str = Field(description="Name of the most possible disease within the given set of diseases")
    reasons: List[DiagnosisReason] = Field(description="A list of reasoning categories and the associated reasons for the final diagnosis")

def doctor_prompt_disease_restricted_ollama(medical_history, diseases, cleaned_similar_cases):
    model = ChatOpenAI(model="gpt-4o-mini")

    prompt_template = f"""
    You are an experienced doctor, and you will be provided with the medical history of a patient containing past medical history,
    physical examination, laboratory examination, and imaging examination results. 
    Patient's medical history: {medical_history}.
    
    Your task is to identify the most likely disease of the patient using differential diagnosis from the these diseases: {diseases}.
    
    Following similar cases maybe helpful {{cleaned_similar_cases}}.

    END OF SIMILAR CASES.
    Please format your response as a JSON object with the following fields:
    - final_diagnosis: Name of the most possible disease within the above set of diseases.
    - reasons: A list of categories (e.g., medical-history, Physical-Examination) with associated reasons for the final diagnosis. Each reason should be precise and brief.

    JSON output:"""

    prompt = ChatPromptTemplate.from_template(prompt_template)

    output_parser = JsonOutputParser(pydantic_object=FinalDiagnosis)

    chain = prompt | model | output_parser
    output = chain.invoke(
        {
            "medical_history": medical_history,
            "diseases": diseases,
            "cleaned_similar_cases": cleaned_similar_cases,
        }
    )
    return output

In [ ]:
sample_diseases = ["parkinson disease", "Vascular Parkinson's Syndrome", "Progressive Supranuclear Palsy"]

In [ ]:
sample_diseases=[
    "Alzheimer's Disease",
    "Parkinson's Disease",
    "Amyotrophic Lateral Sclerosis (ALS)",
    "Huntington's Disease",
    "Multiple System Atrophy (MSA)",
    "Frontotemporal Dementia (FTD)",
    "Vascular Parkinsonism",
    "Multiple Sclerosis (MS)",
    "Guillain-Barré Syndrome (GBS)",
    "Chronic Inflammatory Demyelinating Polyneuropathy (CIDP)",
    "Neuromyelitis Optica (NMO)",
    "Progressive Multifocal Leukoencephalopathy (PML)",
    "Myasthenia Gravis",
    "Duchenne Muscular Dystrophy",
    "Spinal Muscular Atrophy (SMA)",
    "Lambert-Eaton Myasthenic Syndrome",
    "Polymyositis",
    "Dermatomyositis",
    "Stroke",
    "Transient Ischemic Attack (TIA)",
    "Cerebral Aneurysm",
    "Arteriovenous Malformation (AVM)",
    "Cerebral Venous Sinus Thrombosis (CVST)",
    "Essential Tremor",
    "Dystonia",
    "Tardive Dyskinesia",
    "Tourette Syndrome",
    "Ataxia",
    "Restless Legs Syndrome (RLS)",
    "Diabetic Neuropathy",
    "Carpal Tunnel Syndrome",
    "Charcot-Marie-Tooth Disease",
    "Hereditary Neuropathy with Liability to Pressure Palsies (HNPP)",
    "Trigeminal Neuralgia",
    "Meningitis",
    "Encephalitis",
    "Brain Abscess",
    "Creutzfeldt-Jakob Disease (CJD)",
    "Neurosyphilis",
    "Epilepsy",
    "Status Epilepticus",
    "Absence Seizures",
    "Temporal Lobe Epilepsy",
    "Juvenile Myoclonic Epilepsy (JME)",
    "Autism Spectrum Disorder (ASD)",
    "Attention-Deficit/Hyperactivity Disorder (ADHD)",
    "Cerebral Palsy",
    "Rett Syndrome",
    "Tuberous Sclerosis Complex (TSC)",
    "Complex Regional Pain Syndrome (CRPS)",
    "Fibromyalgia",
    "Migraine",
    "Cluster Headache",
    "Postherpetic Neuralgia"
]

In [ ]:
new_summary="""
Patient Basic Information:Elderly female, 80 years old.,
Chief Complaint:Memory decline for 1 year, worsening for more than 10 days.,
Medical History:The patient's memory started to decline without any obvious cause a year ago, manifested as an inability to recall recent events. These symptoms have been progressing and worsening. More than 10 days ago, family members noticed a significant worsening of symptoms, accompanied by a decline in executive function. The patient has a history of hypertension, coronary heart disease, post-coronary stent implantation, and cerebral infarction.,
Physical Examination:Heart, lungs, and abdomen are normal. Neurological System: The patient is conscious, but reaction is slow, understanding and orientation are reduced, memory and calculation ability are decreased, speech is fluent, and she cooperates with the examination. Bilateral nasolabial folds are symmetrical, tongue protrudes in the middle, muscle tone and tendon reflexes of the four limbs are symmetrical and normal, muscle strength is grade V, sensation and coordinated movements are normal, bilateral Babinski sign is negative. Neck is soft, Kernig's sign is negative. NIHSS score: 0 points. Wartenberg's drinking water test: Grade 1. mRS score: 1 point. ADL score: 100 points.,
Imaging Examination:MRI scan: 1. Multiple infarctions in the bilateral semioval center, periventricular, basal ganglia, brainstem, and right cerebellum, and degeneration of brain white matter; 2. Abnormal signal in the left frontal plate, considering epidermoid cyst; 3. Bilateral hippocampal volume reduction; 4. Cerebral arteriosclerosis; 5. Multiple stenosis in the bilateral middle cerebral artery M1 segment, right middle cerebral artery M2 and M3 segments, and bilateral posterior cerebral artery P2 segments. Please combine with clinical history and related examinations.\n(2),
Laboratory Examination:Thyroid function test: Normal.\nCerebrospinal fluid test: 1. Color: Colorless; 2. Transparency: Slightly turbid; 3. Coagulability: Non-coagulating; 4. Pandy's test: Positive (+); 5. Cerebrospinal fluid protein (dry method) (csfpro) 737mg\/L \u2191; 6. Chloride (dry method) (CL) 118.0mmol\/L \u2193; 7. Cerebrospinal fluid immunoglobulin G (CSFIgG) 133.90mg\/L \u2191.\n(3)
"""

In [ ]:
result = doctor_prompt_disease_restricted_ollama(
    medical_history=new_summary,
    diseases=sample_diseases,
    cleaned_similar_cases=cleaned_similar_cases
)
print(result)

In [ ]:
import json
import os

In [ ]:
import json
import os

first_folder="results-gpt-llama-gemma/check_others_input_without_mine"
second_folder= "results5/check_others_input_without_mine/gpt-4o"
third_folder= "results-gpt-llama-gemma/check_others_input_without_mine"
# fourth_folder= "results-self-refinement/combined"

combined_data = {}
folder_files= os.listdir(first_folder)
for file in folder_files:
    first_file_path= f"{first_folder}/{file}"
    second_file_path= f"{second_folder}/{file}"
    third_file_path= f"{third_folder}/{file}"
    # fourth_file_path= f"{fourth_folder}/{file}"
    with open(first_file_path, 'r') as f:
        data1= json.load(f)
    with open(second_file_path, 'r') as f:
        data2 = json.load(f)
    # with open(third_file_path, 'r') as f:
    #     data3 = json.load(f)
    for key in data1.keys():
        data1[key]["predictions"]["gpt-4o"]=data2[key]["predictions"]["gpt-4o"]
        # data1[key]["predictions"]["mistral-nemo"]=data3[key]["predictions"]["mistral-nemo"]
    # # Save the combined dictionary to a new JSON file
    with open(third_file_path, 'w') as out_file:
        json.dump(data1, out_file, indent=4)

# print("Combined JSON saved to 'combined_output.json'")


In [33]:
import difflib

def calculate_word_similarity(word1, word2):
    # Use SequenceMatcher to calculate similarity
    return difflib.SequenceMatcher(None, word1, word2).ratio() * 100   



In [221]:
all_results={}
for folder in os.listdir("results-combined\combination"):
    path=f"results-combined/combination/{folder}/analysis/check_others_input.json"
    try:
        with open(path, "r") as file:
            data = json.load(file)
    except:
        continue
    
    # if folder="results-gpt4o-llama3.1-mistral-nemo":
    #     break 
    departments=list(data.keys())
    final={}

    for department in departments:
        # if department not in ["cardiac surgical department","hepatobiliary and pancreas surgical department"]:
        #     continue
        department_dict=data[department]
        final[department]={"models":{},"all":[],"all_same_failed_possible":[],"all_same_failed":[],"hellucination":[],"hellucination_detected_cases":[],"all_failed_but_different_answers":[]}
        for model in department_dict:
            print(department,model)
            final[department]["models"][model]=department_dict[model]["false"]
            final[department]["all"].extend(department_dict[model]["false"])
            if len(department_dict[model]["false"])==0:
                final[department]["all_same_failed_possible"]=[]
                break
            elif len(final[department]["all_same_failed_possible"])==0:
                final[department]["all_same_failed_possible"]=department_dict[model]["false"]
            else:
                final[department]["all_same_failed_possible"]=list(set(final[department]["all_same_failed_possible"]) & set(department_dict[model]["false"]))
                if len(final[department]["all_same_failed_possible"])==0:
                    break
            # print(final[department]["all_same_failed_possible"])
        final[department]["all_same_failed_possible"].sort()
        final[department]["all"]=list(set(final[department]["all"]))

    for department in departments:
        department_dict=data[department]
        all=final[department]["all_same_failed_possible"]
        models=list(final[department]["models"].keys())
        for case_id in all:
            predicted=[]
            for model in models:
                for case in department_dict[model]["results"]:
                    if case_id == case["case_id"]:
                        predicted.append(case["predicted"])
                        break
            # print(case_id,predicted)
            all_failed=True
            for num in range(len(predicted)-1):
                similarity_percentage = calculate_word_similarity(predicted[num], predicted[num+1])
                if similarity_percentage<90:
                    all_failed=False
                    break            
            if all_failed==True:
                final[department]["all_same_failed"].append(case_id)
            else:
                final[department]["all_failed_but_different_answers"].append(case_id)
    total_all_failed=0
    total_not_all_wrong=0
    total_number_of_hellucination_detected_cases=0
    total_all_failed_but_different_answers=0
    for department in departments:
        department_dict=data[department]
        models=list(final[department]["models"].keys())
        for model  in models:
            cases= department_dict[model]["results"]
            failed_cases_id=department_dict[model]["false"]
            predicted=[]
            for case_id in failed_cases_id:
                for case in cases:
                    if case_id == case["case_id"]:
                        predicted= case["predicted"]
                        all_differential= case["differential-diagnosis"]
                        break
                # print(case_id,predicted)
                hellucination=True
                for differential in all_differential:
                    similarity_percentage = calculate_word_similarity(predicted,differential)
                    if similarity_percentage>=80:
                        hellucination=False
                        break            
                if hellucination==True:
                    final[department]["hellucination"].append([model,case_id])
                    final[department]["hellucination_detected_cases"].append(case_id)
        
        
        final[department]["number_of_all_same_failed_cases"]=len(set(final[department]["all_same_failed"]))
        final[department]["number_of_all_failed_but_different_answers"]=len(set(final[department]["all_failed_but_different_answers"]))
        final[department]["number_not_all_wrong"]=len(final[department]["all"])-len(final[department]["all_same_failed_possible"])
        final[department]["number_of_hellucination_detected_cases"]=len(set(final[department]["hellucination_detected_cases"]))
        
        total_all_failed+=final[department]["number_of_all_same_failed_cases"]
        total_all_failed_but_different_answers+=final[department]["number_of_all_failed_but_different_answers"]
        total_not_all_wrong+=final[department]["number_not_all_wrong"]
        total_number_of_hellucination_detected_cases+=final[department]["number_of_hellucination_detected_cases"]
        # del final[department]["all_same_failed_possible"]
        del final[department]["hellucination_detected_cases"]
        # del final[department]["all"]
        # del final[department]["models"]
        del final[department]["hellucination"]
        # del final[department]["all_same_failed"]
        # del final[department]["all_same_failed_possible"]
        # del final[department]["all_failed_but_different_answers"]
    all_results[folder]={"all_failed":total_all_failed,"all_failed_with_different_answers":total_all_failed_but_different_answers,"not_all_failed":total_not_all_wrong,
                         "cases with hellucinated answers":total_number_of_hellucination_detected_cases
                         }  
    # if folder=="results-gpt4o-llama3.1-mistral-nemo":
    #     break 

respiratory medicine department gemma2
respiratory medicine department mistral-nemo
nephrology department gemma2
nephrology department mistral-nemo
hematology department gemma2
hematology department mistral-nemo
gastrointestinal surgical department gemma2
gastrointestinal surgical department mistral-nemo
gynecology department gemma2
gynecology department mistral-nemo
urinary surgical department gemma2
urinary surgical department mistral-nemo
cardiac surgical department gemma2
gastroenterology department gemma2
gastroenterology department mistral-nemo
hepatobiliary and pancreas surgical department gemma2
hepatobiliary and pancreas surgical department mistral-nemo
endocrinology department gemma2
endocrinology department mistral-nemo
neurology department gemma2
neurology department mistral-nemo
pediatrics department gemma2
pediatrics department mistral-nemo
respiratory medicine department gpt-4o
respiratory medicine department gemma2
nephrology department gpt-4o
nephrology department gemm

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\jzlco\AppData\Local\Temp\ipykernel_5896\3102516694.py:2: SyntaxWarning: invalid escape sequence '\c'
  for folder in os.listdir("results-combined\combination"):


In [222]:
folder

'results-llama3.1-mistral-nemo'

In [223]:
final

{'respiratory medicine department': {'models': {'mistral-nemo': ['202',
    '205',
    '208',
    '217',
    '224'],
   'llama3.1': ['202', '217', '224']},
  'all': ['217', '224', '202', '205', '208'],
  'all_same_failed_possible': ['202', '217', '224'],
  'all_same_failed': ['202', '217', '224'],
  'all_failed_but_different_answers': [],
  'number_of_all_same_failed_cases': 3,
  'number_of_all_failed_but_different_answers': 0,
  'number_not_all_wrong': 2,
  'number_of_hellucination_detected_cases': 1},
 'nephrology department': {'models': {'mistral-nemo': ['1151',
    '1154',
    '1157',
    '1160',
    '1190'],
   'llama3.1': ['1151', '1157', '1190']},
  'all': ['1157', '1154', '1190', '1151', '1160'],
  'all_same_failed_possible': ['1151', '1157', '1190'],
  'all_same_failed': ['1151', '1190'],
  'all_failed_but_different_answers': ['1157'],
  'number_of_all_same_failed_cases': 2,
  'number_of_all_failed_but_different_answers': 1,
  'number_not_all_wrong': 2,
  'number_of_hellucinat

In [224]:
import pandas as pd
df = pd.DataFrame.from_dict(all_results, orient='index')

In [225]:
all_results

{'results-gemma2-mistral-nemo': {'all_failed': 35,
  'all_failed_with_different_answers': 13,
  'not_all_failed': 39,
  'cases with hellucinated answers': 22},
 'results-gpt4o-gemma2': {'all_failed': 21,
  'all_failed_with_different_answers': 6,
  'not_all_failed': 43,
  'cases with hellucinated answers': 5},
 'results-gpt4o-gemma2-mistral-nemo': {'all_failed': 17,
  'all_failed_with_different_answers': 9,
  'not_all_failed': 49,
  'cases with hellucinated answers': 13},
 'results-gpt4o-llama3.1': {'all_failed': 26,
  'all_failed_with_different_answers': 5,
  'not_all_failed': 31,
  'cases with hellucinated answers': 2},
 'results-gpt4o-llama3.1-gemma2': {'all_failed': 24,
  'all_failed_with_different_answers': 5,
  'not_all_failed': 44,
  'cases with hellucinated answers': 10},
 'results-gpt4o-llama3.1-mistral-nemo': {'all_failed': 26,
  'all_failed_with_different_answers': 3,
  'not_all_failed': 39,
  'cases with hellucinated answers': 7},
 'results-gpt4o-mistral-nemo': {'all_failed'

In [226]:
df

,all_failed,all_failed_with_different_answers,not_all_failed,cases with hellucinated answers
results-gemma2-mistral-nemo,35,13,39,22
results-gpt4o-gemma2,21,6,43,5
results-gpt4o-gemma2-mistral-nemo,17,9,49,13
results-gpt4o-llama3.1,26,5,31,2
results-gpt4o-llama3.1-gemma2,24,5,44,10
results-gpt4o-llama3.1-mistral-nemo,26,3,39,7
results-gpt4o-mistral-nemo,25,7,40,17
results-llama-mistral-gemma,38,7,36,16
results-llama3.1-gemma2,40,8,21,7
results-llama3.1-mistral-nemo,38,9,45,22
